# Parameter Initialization

Now that we know how to access the parameters,
let's look at how to initialize them properly.
We discussed the need for proper initialization in :numref:`sec_numerical_stability`.
The deep learning framework provides default random initializations to its layers.
However, we often want to initialize our weights
according to various other protocols. The framework provides most commonly
used protocols, and also allows to create a custom initializer.


# 参数初始化

现在我们已经知道如何访问参数，
让我们看看如何正确地初始化它们。
我们曾在 :numref:`sec_numerical_stability` 中讨论过良好初始化的必要性。
深度学习框架提供默认随机初始化给其各层使用。
然而，我们经常需要根据其他协议
来初始化权重。深度学习框架提供了最常用的
协议，同时也允许创建自定义初始化方法。

In [1]:
import torch
from torch import nn

By default, PyTorch initializes weight and bias matrices
uniformly by drawing from a range that is computed according to the input and output dimension.
PyTorch's `nn.init` module provides a variety
of preset initialization methods.


默认情况下，PyTorch 会通过从根据输入和输出维度计算出的范围中均匀采样来初始化权重矩阵和偏置向量。PyTorch 的 `nn.init` 模块提供了多种预置的初始化方法。

In [3]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## [**Built-in Initialization**]

Let's begin by calling on built-in initializers.
The code below initializes all weight parameters
as Gaussian random variables
with standard deviation 0.01, while bias parameters are cleared to zero.


## [**内置初始化**]

我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为
标准差为0.01的高斯随机变量，
且将偏置参数清零。

In [4]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0085,  0.0173,  0.0255,  0.0021]), tensor(0.))

In [12]:
def chi_init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(chi_init_normal)
net[0].weight.data,net[0].bias.data

(tensor([[ 0.0022, -0.0096, -0.0070,  0.0105],
         [ 0.0186,  0.0033, -0.0124, -0.0104],
         [ 0.0027,  0.0133, -0.0073,  0.0072],
         [ 0.0244,  0.0050,  0.0017, -0.0213],
         [-0.0221, -0.0137, -0.0076,  0.0006],
         [ 0.0201, -0.0079, -0.0069,  0.0144],
         [ 0.0108,  0.0065, -0.0088, -0.0059],
         [ 0.0066, -0.0119, -0.0133, -0.0240]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

We can also initialize all the parameters
to a given constant value (say, 1).


我们也可以将所有参数初始化为给定的常数值（例如1）。

In [14]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [13]:
def chi_init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

[**We can also apply different initializers for certain blocks.**]
For example, below we initialize the first layer
with the Xavier initializer
and initialize the second layer
to a constant value of 42.


[**我们也可以对某些块应用不同的初始化方法。**]
例如在下面的例子中，我们使用Xavier初始化器初始化第一层，
并将第二层初始化为常数值42。

In [16]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([-0.6069, -0.4927,  0.2211,  0.5363])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


### [**Custom Initialization**]

Sometimes, the initialization methods we need
are not provided by the deep learning framework.
In the example below, we define an initializer
for any weight parameter $w$ using the following strange distribution:

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \textrm{ with probability } \frac{1}{4} \\
            0    & \textrm{ with probability } \frac{1}{2} \\
        U(-10, -5) & \textrm{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们为任意权重参数$w$定义了一个基于以下特殊分布的初始化器：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \textrm{ 以 } \frac{1}{4} \textrm{ 的概率} \\
            0    & \textrm{ 以 } \frac{1}{2} \textrm{ 的概率} \\
        U(-10, -5) & \textrm{ 以 } \frac{1}{4} \textrm{ 的概率}
    \end{cases}
\end{aligned}
$$

Again, we implement a `my_init` function to apply to `net`.


In [20]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-7.9225, -0.0000,  7.9086,  0.0000],
        [ 6.7229, -0.0000,  8.6296,  8.6200]], grad_fn=<SliceBackward0>)

Note that we always have the option
of setting parameters directly.


In [21]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  8.9086,  1.0000])

## Summary

We can initialize parameters using built-in and custom initializers.

## Exercises

Look up the online documentation for more built-in initializers.


## 小结

我们可以使用内置初始化方法和自定义初始化方法来初始化模型参数。

## 练习

查阅在线文档，了解更多可用的内置初始化方法。

[Discussions](https://discuss.d2l.ai/t/8090)
